# Project task 02: Restaurant recommendation

In [ ]:
import scipy.sparse as sp
import numpy as np
from scipy.sparse.linalg import svds
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.linear_model import LinearRegression, Ridge
import time

The goal of this task is to recommend restaurants to users based on the rating data in the Yelp dataset. For this, we try to predict the rating a user will give to a restaurant they have not been to yet based on a latent factor model.

## 1. Load data

First download `ratings.npy` from Piazza ([download link](https://syncandshare.lrz.de/dl/fiKMoxRNusLoFpFHkXXEgvdZ/ratings.npy)).

In [ ]:
ratings = np.load("ratings.npy")

In [ ]:
# We have triplets of (user, restaurant, rating).
ratings

Now we transform the data into a matrix of dimension [N, D], where N is the number of users and D is the number of restaurants in the dataset.  
We **strongly recommend** to load the data as a sparse matrix to avoid out-of-memory issues.

In [ ]:
# Store the matrix into the variable M

### YOUR CODE HERE ### ===> DONE
M = sp.csr_matrix((ratings[:,2], (ratings[:,0], ratings[:,1])))

In [ ]:
M

## 2. Preprocess the data

In the preprocessing step, we recursively remove all users and restaurants with 10 or less ratings. 

Then, we randomly select 200 data points for the validation and test sets, respectively.

After this, we subtract the mean rating for each users to account for this global effect.   
**Hint**: Some entries might become zero in this process -- but these entries are different than the 'unknown' zeros in the matrix. Store the indices of which we have data available in a separate variable.

In [ ]:
def cold_start_preprocessing(matrix, min_entries):
    """
    Recursively removes rows and columns from the input matrix which have less than min_entries nonzero entries.
    
    Parameters
    ----------
    matrix      : sp.spmatrix, shape [N, D]
                  The input matrix to be preprocessed.
    min_entries : int
                  Minimum number of nonzero elements per row and column.

    Returns
    -------
    matrix      : sp.spmatrix, shape [N', D']
                  The pre-processed matrix, where N' <= N and D' <= D
        
    """
    print("Shape before: {}".format(matrix.shape))
    
    ### YOUR CODE HERE ### ===> DONE
    
    #Preprocessing is done by applying the recursive function preproc_matrix from the task_02_func.py file
    from task_02_func import preproc_matrix   
    
    matrix = preproc_matrix(matrix, min_entries)[0]

    nnz = matrix>0
    assert (nnz.sum(0).A1 > min_entries).all()
    assert (nnz.sum(1).A1 > min_entries).all()
    
    print("Shape after: {}".format(matrix.shape))
    
    return matrix

In [ ]:
def shift_user_mean(matrix):
    """
    Subtract the mean rating per user from the non-zero elements in the input matrix.
    
    Parameters
    ----------
    matrix : sp.spmatrix, shape [N, D]
             Input sparse matrix.
    Returns
    -------
    matrix : sp.spmatrix, shape [N, D]
             The modified input matrix.
    
    user_means : np.array, shape [N, 1]
                 The mean rating per user that can be used to recover the absolute ratings from the mean-shifted ones.

    """
    
    ### YOUR CODE HERE ### ===> DONE
    
    #Compute the mean for each row (user) considering only nonzero elements
    user_means = np.array([matrix.sum(axis = 1).A1 / matrix.getnnz(axis = 1)]).T
    
    #Create a matrix where each nonzero entry from the original matrix 
    #is replaced by its row's mean
    matrix_means = sp.diags(np.ravel(user_means)).dot(matrix.sign())
    
    #Substracting this matrix from the original one makes sure 
    #that only nonzero entries are shifted
    matrix = matrix - matrix_means
    
    assert np.all(np.isclose(matrix.mean(1), 0))
    return matrix, user_means

In [ ]:
def split_data(matrix, n_validation, n_test):
    """
    Extract validation and test entries from the input matrix. 
    
    Parameters
    ----------
    matrix          : sp.spmatrix, shape [N, D]
                      The input data matrix.
    n_validation    : int
                      The number of validation entries to extract.
    n_test          : int
                      The number of test entries to extract.

    Returns
    -------
    matrix_split    : sp.spmatrix, shape [N, D]
                      A copy of the input matrix in which the validation and test entries have been set to zero.
    
    val_idx         : tuple, shape [2, n_validation]
                      The indices of the validation entries.
    
    test_idx        : tuple, shape [2, n_test]
                      The indices of the test entries.
    
    val_values      : np.array, shape [n_validation, ]
                      The values of the input matrix at the validation indices.
                      
    test_values     : np.array, shape [n_test, ]
                      The values of the input matrix at the test indices.

    """
    
    ### YOUR CODE HERE ### ==> DONE
    
    #Get the indices of all nonzero elements in the matrix
    (row_nonzero, col_nonzero) = M.nonzero()
    #Sample the indices for validation and test data together from all nonzero elements
    #(by this avoid accidentally picking data point for validation and test at the same time)
    rand_idx = np.random.choice(len(row_nonzero), n_validation + n_test, replace = False)
    #Split validation and test indices
    rand_idx_val = rand_idx[:n_validation]
    val_idx = (row_nonzero[rand_idx_val], col_nonzero[rand_idx_val])
    rand_idx_test = rand_idx[n_validation:]
    test_idx = (row_nonzero[rand_idx_test], col_nonzero[rand_idx_test])
    
    #Get the values of the validation and the test data  
    val_values = np.ravel(matrix[val_idx])
    test_values = np.ravel(matrix[test_idx])
    #Make a copy of the input matrix in which the validation and test entries are set to zero
    matrix_split = matrix
    matrix_split[val_idx] = 0
    matrix_split[test_idx] = 0
    
    matrix_split.eliminate_zeros()
    return matrix_split, val_idx, test_idx, val_values, test_values

In [ ]:
M = cold_start_preprocessing(M, 10)

In [ ]:
n_validation = 200
n_test = 200
# Split data
M_train, val_idx, test_idx, val_values, test_values = split_data(M, n_validation, n_test)

In [ ]:
# Store away the nonzero indices of M before subtracting the row means.
nonzero_indices =  M_train.nonzero() ### YOUR CODE HERE ### ==> DONE
# Remove user means.
M_shifted, user_means = shift_user_mean(M_train)

# Apply the same shift to the validation and test data.
val_values_shifted = np.ravel(np.array([val_values[idx] - user_means[val_idx[0][idx]] \
                               for idx in range(len(val_values))])) ### YOUR CODE HERE ### ==> DONE
test_values_shifted = np.ravel(np.array([test_values[idx] - user_means[test_idx[0][idx]] \
                                for idx in range(len(test_values))])) ### YOUR CODE HERE ### ==> DONE

## 3. Alternating optimization

In the first step, we will approach the problem via alternating optimization, as learned in the lecture.

In [ ]:
def initialize_Q_P(matrix, k, init='random'):
    """
    Initialize the matrices Q and P for a latent factor model.
    
    Parameters
    ----------
    matrix : sp.spmatrix, shape [N, D]
             The matrix to be factorized.
    k      : int
             The number of latent dimensions.
    init   : str in ['svd', 'random'], default: 'random'
             The initialization strategy. 'svd' means that we use SVD to initialize P and Q, 'random' means we initialize
             the entries in P and Q randomly in the interval [0, 1).

    Returns
    -------
    Q : np.array, shape [N, k]
        The initialized matrix Q of a latent factor model.

    P : np.array, shape [k, D]
        The initialized matrix P of a latent factor model.
    """

    
    if init == 'svd':
        ### YOUR CODE HERE ### ==> DONE
        u, s, vt = svds(matrix, k)
        #in our problem we havethe form Q = UE, so we multiply s into u
        Q = u.dot(np.diag(s))
        P = vt
    elif init == 'random':
        ### YOUR CODE HERE ### ==> DONE
        Q = np.random.rand(matrix.shape[0], k)
        P = np.random.rand(k, matrix.shape[1])
    else:
        raise ValueError
        
    assert Q.shape == (matrix.shape[0], k)
    assert P.shape == (k, matrix.shape[1])
    return Q, P

In [ ]:
def get_matrix_update(matrix_M, matrix_fixed, matrix_to_be_updated, nonzero_n, nonzero_m, reg_strength):
    #assumption        matrix_M = matrix_fixed * matrix_to_be_updated
    #dimensions:        n x m         n x d             d x m
    #in case of P update: n = numbers of users,       m = numbers of restaurants
    #in case of P update: n = numbers of restaurants, m = numbers of users
    #Matrix with collected optimal updates
    
    matrix_optimal = np.zeros((matrix_to_be_updated.shape[0], matrix_to_be_updated.shape[1]))

    #ridge regression with closed form solution
    model = Ridge(alpha=reg_strength)
    matrix_M_T = sp.csr_matrix(matrix_M.transpose())

    for counter in range(matrix_to_be_updated.shape[1]):
        #find users that correspond to the item        
        relevant_indices = nonzero_n[nonzero_m == counter]        
        #solution with solver
        abg = matrix_M_T[nonzero_m[nonzero_m == counter], relevant_indices].A1
        model.fit(matrix_fixed[relevant_indices,:], abg)
        matrix_optimal[:,counter] = model.coef_.T
        
    return matrix_optimal

In [ ]:
def loss_calculation(values, indices, Q, P):

    #build matrix with guessed ratings from Q and P
    M_guess = Q.dot(P)
    #compute difference between real values and guessed values
    diff = values - M_guess[indices]
    #compute loss as L2**2 norm
    loss = np.linalg.norm(diff) ** 2

    return loss

In [ ]:
def latent_factor_alternating_optimization(M, non_zero_idx, k, val_idx, val_values,
                                           reg_lambda, max_steps=100, init='random',
                                           log_every=1, patience=10, eval_every=1):
    """
    Perform matrix factorization using alternating optimization. Training is done via patience,
    i.e. we stop training after we observe no improvement on the validation loss for a certain
    amount of training steps. We then return the best values for Q and P oberved during training.
    
    Parameters
    ----------
    M                 : sp.spmatrix, shape [N, D]
                        The input matrix to be factorized.
                      
    non_zero_idx      : np.array, shape [nnz, 2]
                        The indices of the non-zero entries of the un-shifted matrix to be factorized. 
                        nnz refers to the number of non-zero entries. Note that this may be different
                        from the number of non-zero entries in the input matrix M, e.g. in the case
                        that all ratings by a user have the same value.
    
    k                 : int
                        The latent factor dimension.
    
    val_idx           : tuple, shape [2, n_validation]
                        Tuple of the validation set indices.
                        n_validation refers to the size of the validation set.
                      
    val_values        : np.array, shape [n_validation, ]
                        The values in the validation set.
                      
    reg_lambda        : float
                        The regularization strength.
                      
    max_steps         : int, optional, default: 100
                        Maximum number of training steps. Note that we will stop early if we observe
                        no improvement on the validation error for a specified number of steps
                        (see "patience" for details).
                      
    init              : str in ['random', 'svd'], default 'random'
                        The initialization strategy for P and Q. See function initialize_Q_P for details.
    
    log_every         : int, optional, default: 1
                        Log the training status every X iterations.
                    
    patience          : int, optional, default: 10
                        Stop training after we observe no improvement of the validation loss for X evaluation
                        iterations (see eval_every for details). After we stop training, we restore the best 
                        observed values for Q and P (based on the validation loss) and return them.
                      
    eval_every        : int, optional, default: 1
                        Evaluate the training and validation loss every X steps. If we observe no improvement
                        of the validation error, we decrease our patience by 1, else we reset it to *patience*.

    Returns
    -------
    best_Q            : np.array, shape [N, k]
                        Best value for Q (based on validation loss) observed during training
                      
    best_P            : np.array, shape [k, D]
                        Best value for P (based on validation loss) observed during training
                      
    validation_losses : list of floats
                        Validation loss for every evaluation iteration, can be used for plotting the validation
                        loss over time.
                        
    train_losses      : list of floats
                        Training loss for every evaluation iteration, can be used for plotting the training
                        loss over time.                     
    
    converged_after   : int
                        it - patience*eval_every, where it is the iteration in which patience hits 0,
                        or -1 if we hit max_steps before converging. 

    """

    ### YOUR CODE HERE ### ===> DONE
    
    #Initialization of Q and P
    Q, P = initialize_Q_P(M, k, init)
    
    #init lists
    train_losses = []
    validation_losses = []
    list_Qs = []
    list_Ps = []
    
    #temporary patience that is decreased when validation loss stays from iteration to iteration
    new_patience = patience
    
    start_time = time.time()
    
    #initial training data loss
    init_train_loss = loss_calculation(M[non_zero_idx], non_zero_idx, Q, P)
    init_validation_loss = loss_calculation(val_values, val_idx, Q, P)
    print("Initial training loss: {0:.3f}, initial validation loss: {1:.3f}"\
          .format(init_train_loss, init_validation_loss))
    
    #iterate until max_steps reached
    for step in range(max_steps):
        
        #alternating update of P and Q matrix
        P = get_matrix_update(M, Q, P, non_zero_idx[0], non_zero_idx[1], reg_lambda)
        Q = get_matrix_update(sp.csr_matrix(M.transpose()), P.T, Q.T, non_zero_idx[1], non_zero_idx[0], reg_lambda).T
               
        #evaluate losses every eval_every steps
        if step % eval_every == 0:
            
            #add P and Q matrix to list for later choosing of best matrices
            list_Qs.append(Q)
            list_Ps.append(P)
            
            #calculate training loss
            train_loss = loss_calculation(M[non_zero_idx], non_zero_idx, Q, P)
            #add loss to list for e.g. plotting
            train_losses.append(train_loss)
            
            #calculate validation loss
            validation_loss = loss_calculation(val_values, val_idx, Q, P)
            #add loss to list for choosing of iteration with best loss and e.g. plotting
            validation_losses.append(validation_loss)
            
            #print losses every log_every iterations
            if step % log_every == 0:
            
                print("Iteration {0}, training loss: {1:.3f}, validation loss: {2:.3f}"\
                      .format(step + 1, train_loss, validation_loss))
                
            try:
                #check if validation loss improves
                #if not, decrease temporary patience by 1
                if (validation_loss >= best_val_loss) or np.isclose(validation_loss, best_val_loss):
                    
                    new_patience -= 1
                    
                    #if temporary patience reaches 0 --> stop optimization
                    #and return parameters
                    if new_patience <= 0:
                        
                        end_time = time.time()
                        
                        converged_after = step - patience*eval_every + 1
                        
                        print("Converged after {0} iterations, on average {1:.3f} per iteration"\
                              .format(converged_after,(end_time - start_time)/step))
                        return best_Q, best_P, validation_losses, train_losses, converged_after
                
                else:
                    #if it improves, reset patience to original value
                    new_patience = patience
                
                    #save validation loss for comparison in next steps
                    best_val_loss = validation_loss
                    #save best Q and P
                    best_Q = Q
                    best_P = P
            
            #if previous validation loss does not exist (first iteration)
            except NameError:
                #save validation loss for comparison in next step
                best_val_loss = validation_loss
                #save Q and P as initial best
                best_Q = Q
                best_P = P
    
    
    #after maximum number of steps is reached return parameters
    
    end_time = time.time()
    
    converged_after = -1
    
    print("Ran into maximum of {0} steps, on average {1:.3f} per iteration"\
          .format(max_steps,(end_time - start_time)/max_steps))
    
    return best_Q, best_P, validation_losses, train_losses, converged_after


#### Train the latent factor model with alternating optimization.

a) Learn the optimal $P$ and $Q$ using alternating optimization. That is, during each iteration you first update $Q$ while having $P$ fixed and then vice versa. Run the alternating optimization algorithm with $k=100$ and $\lambda=1$. Plot the training and validation losses over time.

In [ ]:
Q_a, P_a, val_l_a, tr_l_a, conv_a = latent_factor_alternating_optimization(M_shifted, nonzero_indices, 
                                                                           k=100, val_idx=val_idx,
                                                                           val_values=val_values_shifted, 
                                                                           reg_lambda=1, init='random',
                                                                           max_steps=100, patience=10)

#### Plot the validation and training losses over (training) time

In [ ]:
### YOUR PLOTTING CODE HERE ### ==> DONE

plt.plot(range(len(val_l_a)), val_l_a)
plt.xticks(np.arange(len(val_l_a), step=2))
plt.xlabel('Training iteration')
plt.ylabel('Validation loss')
plt.title('Alternating optimization validation loss, k = 100')
plt.show()

plt.plot(range(len(tr_l_a)), tr_l_a)
plt.xticks(np.arange(len(tr_l_a), step=2))
plt.xlabel('Training iteration')
plt.ylabel('Training loss')
plt.title('Alternating optimization training loss, k = 100')
plt.show()

b) (**Optional**): Try some different latent dimensions $k$ in the range [5, 100]. What do you observe (convergence time, final training/validation losses)?

With decreasing number of latent dimensions $k$ our validation loss increases and the convergence time decreases.

In [ ]:
Q_a_2, P_a_2, val_l_a_2, tr_l_a_2, conv_a_2 = latent_factor_alternating_optimization(M_shifted, nonzero_indices, 
                                                                           k=20, val_idx=val_idx,
                                                                           val_values=val_values_shifted, 
                                                                           reg_lambda=0.1, init='random',
                                                                           max_steps=100, patience=10)

#### Plot the validation and training losses over (training) time

In [ ]:
### YOUR PLOTTING CODE HERE ### ==> DONE

plt.plot(range(len(val_l_a_2)), val_l_a_2)
plt.xticks(np.arange(len(val_l_a_2), step=2))
plt.xlabel('Training iteration')
plt.ylabel('Validation loss')
plt.title('Alternating optimization validation loss, k = 20')
plt.show()

plt.plot(range(len(tr_l_a_2)), tr_l_a_2)
plt.xticks(np.arange(len(tr_l_a_2), step=2))
plt.xlabel('Training iteration')
plt.ylabel('Training loss')
plt.title('Alternating optimization training loss, k = 20')
plt.show()

## 4. Latent factorization using gradient descent

We now use gradient descent to factorize our ratings matrix. We will try both (mini-) batch and stochastic gradient descent. You can use the following equations for your implementation.

Recall that the objective function (loss) we wanted to optimize was:
$$
\mathcal{L} = \min_{P, Q} \sum_{(x, i) \in W} (r_{xi} - \mathbf{q}_i^T\mathbf{p}_x)^2 + \lambda_1\sum_x{\left\lVert \mathbf{p}_x  \right\rVert}^2 + \lambda_2\sum_i {\left\lVert\mathbf{q}_i  \right\rVert}^2
$$

where $W$ is the set of $(x, i)$ pairs for which $r_{xi}$ is known (in this case our known play counts). Here we have also introduced two regularization terms to help us with overfitting where $\lambda_1$ and $\lambda_2$ are hyper-parameters that control the strength of the regularization.

Naturally optimizing with gradient descent involves computing the gradient of the loss function $\mathcal{L}$ w.r.t. to the parameters. To help you solve the task we provide the following:

$$
\frac{\partial ((r_{xi} - \mathbf{q}_i^T\mathbf{p}_x)^2)}{\partial \mathbf{p}_x} = -2(r_{xi} - \mathbf{q}_i^T\mathbf{p}_x)\mathbf{q}_i\;, ~~~
\frac{\partial ((r_{xi} - \mathbf{q}_i^T\mathbf{p}_x)^2)}{\partial \mathbf{q}_i} = -2(r_{xi} - \mathbf{q}_i^T\mathbf{p}_x)\mathbf{p}_x 
$$

$$
\frac{\partial(\lambda_1{\left\lVert \mathbf{p}_x \right\rVert}^2)}{\partial \mathbf{p}_x} = 2 \lambda_1 \mathbf{p_x} \;, ~~~
\frac{\partial(\lambda_2{\left\lVert \mathbf{q}_i \right\rVert}^2)}{\partial \mathbf{q}_i} = 2 \lambda_2 \mathbf{q_i}
$$

**Hint**: You have to carefully consider how to combine the given partial gradients depending
on which variants of gradient descent you are using.  
**Hint 2**: It may be useful to scale the updates to $P$ and $Q$ by $\frac{1}{batch\_size}$ (in the case of full-sweep updates, this would be $\frac{1}{n\_users}$ for $Q$ and $\frac{1}{n\_restaurants}$ for $P$).


For each of the gradients descent variants you try report and compare the following:
* How many iterations do you need for convergence.
* Plot the loss (y axis) for each iteration (x axis).


In [ ]:
def update(Q, P, M, nonzero_indices, learning_rate, reg_lambda, scale, k):     
            
        #initialisation of later needed vector or matrices
        sum_over_usergradient = np.zeros((M.shape[0], k))
        sum_over_itemsgradient = np.zeros((M.shape[1], k))
        user_counter = np.zeros(M.shape[0], dtype= int)
        items_counter = np.zeros(M.shape[1], dtype= int)
        
        #get the training data points
        r_xi = M[user_indices,items_indices].A1
        
        #split up tuple into two lists with the indices for the nonzero users and nonzero items
        user_indices, items_indices = (nonzero_indices[0], nonzero_indices[1])
        
        #count how often a specific user/item occurs within the training data for later normalization
        np.add.at(user_counter, user_indices, 1)
        np.add.at(items_counter, items_indices, 1) 
        
        #compute a piece of the gradient that can be applied to users and items equivalently
        gradient_common = -2 * (r_xi - np.sum(Q[user_indices] * P.T[items_indices], axis=1))
        
        #compute the loss part of the gradient separatly for users and items
        gradient_user = gradient_common[:, None] * P.T[items_indices]
        gradient_items = gradient_common[:, None] * Q[user_indices]
        
        #sum up the gradients for each user/item individually
        np.add.at(sum_over_usergradient, user_indices, gradient_user)
        np.add.at(sum_over_itemsgradient, items_indices, gradient_items)
        
        #define a mask in order to get the relevant values for the scaling
        mask_user = (user_counter != 0)
        mask_item = (items_counter != 0)
        
        #scale the computed gradients individual per user/item
        sum_over_usergradient[mask_user] = sum_over_usergradient[mask_user] / user_counter[mask_user, None]
        sum_over_itemsgradient[mask_item] = sum_over_itemsgradient[mask_item] / items_counter[mask_item, None]
   
        #scale the general gradient and add the regularization Parameter
        sum_over_usergradient[mask_user] = scale * sum_over_usergradient[mask_user] + 2 * reg_lambda * Q[mask_user]
        sum_over_itemsgradient[mask_item] = scale * sum_over_itemsgradient[mask_item] + 2 * reg_lambda * P.T[mask_item]
        
        #update the matrices P and Q
        Q = Q - learning_rate * sum_over_usergradient
        P = P - learning_rate * np.transpose(sum_over_itemsgradient)
        
        return Q ,P

In [ ]:
def latent_factor_gradient_descent(M, non_zero_idx, k, val_idx, val_values, 
                                   reg_lambda, learning_rate, batch_size=-1,
                                   max_steps=50000, init='random',
                                   log_every=1000, patience=20,
                                   eval_every=50):
    """
    Perform matrix factorization using gradient descent. Training is done via patience,
    i.e. we stop training after we observe no improvement on the validation loss for a certain
    amount of training steps. We then return the best values for Q and P oberved during training.
    
    Parameters
    ----------
    M                 : sp.spmatrix, shape [N, D]
                        The input matrix to be factorized.
                      
    non_zero_idx      : np.array, shape [nnz, 2]
                        The indices of the non-zero entries of the un-shifted matrix to be factorized. 
                        nnz refers to the number of non-zero entries. Note that this may be different
                        from the number of non-zero entries in the input matrix M, e.g. in the case
                        that all ratings by a user have the same value.
    
    k                 : int
                        The latent factor dimension.
    
    val_idx           : tuple, shape [2, n_validation]
                        Tuple of the validation set indices.
                        n_validation refers to the size of the validation set.
                      
    val_values        : np.array, shape [n_validation, ]
                        The values in the validation set.
                      
    reg_lambda        : float
                        The regularization strength.

    learning_rate     : float
                        Step size of the gradient descent updates.
                        
    batch_size        : int, optional, default: -1
                        (Mini-) batch size. -1 means we perform standard full-sweep gradient descent.
                        If the batch size is >0, use mini batches of this given size.
                        
    max_steps         : int, optional, default: 100
                        Maximum number of training steps. Note that we will stop early if we observe
                        no improvement on the validation error for a specified number of steps
                        (see "patience" for details).
                      
    init              : str in ['random', 'svd'], default 'random'
                        The initialization strategy for P and Q. See function initialize_Q_P for details.
    
    log_every         : int, optional, default: 1
                        Log the training status every X iterations.
                    
    patience          : int, optional, default: 10
                        Stop training after we observe no improvement of the validation loss for X evaluation
                        iterations (see eval_every for details). After we stop training, we restore the best 
                        observed values for Q and P (based on the validation loss) and return them.
                      
    eval_every        : int, optional, default: 1
                        Evaluate the training and validation loss every X steps. If we observe no improvement
                        of the validation error, we decrease our patience by 1, else we reset it to *patience*.
                        
    Returns
    -------
    best_Q            : np.array, shape [N, k]
                        Best value for Q (based on validation loss) observed during training
                      
    best_P            : np.array, shape [k, D]
                        Best value for P (based on validation loss) observed during training
                      
    validation_losses : list of floats
                        Validation loss for every evaluation iteration, can be used for plotting the validation
                        loss over time.
                        
    train_losses      : list of floats
                        Training loss for every evaluation iteration, can be used for plotting the training
                        loss over time.                     
    
    converged_after   : int
                        it - patience*eval_every, where it is the iteration in which patience hits 0,
                        or -1 if we hit max_steps before converging. 

    """
    
    ### YOUR CODE HERE ### ==> DONE
    
    #initialization for Q and P
    Q,P = initialize_Q_P(M, k, init)
    
    #init lists
    train_losses = []
    validation_losses = []
    list_Qs = []
    list_Ps = []
    
    #temporary patience that is decreased when validation loss stays from iteration to iteration
    new_patience = patience
        
    #initial training data loss
    init_train_loss = loss_calculation(M[non_zero_idx], non_zero_idx, Q, P)
    init_vali_loss = loss_calculation(val_values, val_idx, Q, P)
    print("Initial training loss: {0:.3f}, initial validation loss: {1:.3f}".format(init_train_loss, init_vali_loss))
    
    start_time = time.time()
    
    #iterate until max_steps reached
    for step in range(max_steps):
        
        #full sweep
        if batch_size == -1:
            
            #scale with N/N
            scale = 1
            
            #batch is the whole set of nonzero elements
            nonzeros_batch = non_zero_idx
            
        #mini batch    
        else:
            #scale with N/b
            scale = 1 #len(nonzero_indices[0]) / batch_size
            
            #choose batch_size random samples from the non_zero_indices
            batch_idx = np.random.choice(len(nonzero_indices[0]), size = batch_size, replace = False)
            nonzeros_batch = (non_zero_idx[0][batch_idx], non_zero_idx[1][batch_idx])
            
        #gradient descent update
        Q, P = update(Q, P, M, nonzeros_batch, learning_rate, reg_lambda, scale, k)
               
        #evaluate losses every eval_every steps
        if step % eval_every == 0:
            
            #add P and Q matrix to list for later choosing of best matrices
            list_Qs.append(Q)
            list_Ps.append(P)
            
            #calculate training loss
            train_loss = loss_calculation(M[non_zero_idx], non_zero_idx, Q, P)
            #add loss to list for e.g. plotting
            train_losses.append(train_loss)
            
            #calculate validation loss
            validation_loss = loss_calculation(val_values, val_idx, Q, P)
            #add loss to list for choosing of iteration with best loss and e.g. plotting
            validation_losses.append(validation_loss)
            
            #print losses every log_every iterations
            if step % log_every == 0:
            
                print("Iteration {0}, training loss: {1:.3f}, validation loss: {2:.3f}".format(step, train_loss, validation_loss))
            
            try:
                #check if validation loss improves
                #if not, decrease temporary patience by 1
                if (validation_loss >= best_val_loss) or np.isclose(validation_loss, best_val_loss):
                    
                    new_patience -= 1
                    
                    #if temporary patience reaches 0 --> stop optimization
                    #and return parameters
                    if new_patience == 0:
                        
                        end_time = time.time()
                        
                        converged_after = step - patience*eval_every
                        idx_best_val_l = validation_losses.index(min(validation_losses))
                        best_Q = list_Qs[idx_best_val_l]
                        best_P = list_Ps[idx_best_val_l]
                        
                        print("Converged after {0} iterations, on average {1:.3f} per iteration".format(converged_after,(end_time - start_time)/step))
                        return best_Q, best_P, validation_losses, train_losses, converged_after
                
                else:
                    #if it improves, reset patience to original value
                    new_patience = patience
                
                    #save validation loss for comparison in next steps
                    best_val_loss = validation_loss
            
            #if previous validation loss does not exist (first iteration)
            except NameError:
                #save validation loss for comparison in next step
                best_val_loss = validation_loss
    
    
    #after maximum number of steps is reached return parameters
    
    end_time = time.time()
    
    converged_after = -1
    idx_best_val_l = validation_losses.index(min(validation_losses))
    best_Q = list_Qs[idx_best_val_l]
    best_P = list_Ps[idx_best_val_l]
    
    print("Ran into maximum of {0} steps, on average {1:.3f} per iteration".format(max_steps,(end_time - start_time)/max_steps))        
    
            
    return best_Q, best_P, validation_losses, train_losses, converged_after

#### Train the latent factor model with alternating optimization.

a) Learn the optimal $P$ and $Q$ using standard gradient descent. That is, during each iteration you have to use all of the training examples and update $Q$ and $P$ for all users and songs at once. Try the algorithm with $k=30$, $\lambda=1$, and learning rate of 0.1. Initialize $Q$ and $P$ with SVD.  

In [ ]:
Q_g_sweep, P_g_sweep, val_l_g_sweep, tr_l_g_sweep, conv_g_sweep =  latent_factor_gradient_descent(M_shifted, nonzero_indices, 
                                                                                                   k=30, val_idx=val_idx,
                                                                                                   val_values=val_values_shifted, 
                                                                                                   reg_lambda=1e-0, learning_rate=1e-3,
                                                                                                   init='svd', batch_size=-1,
                                                                                                   max_steps=10000, log_every=20, 
                                                                                                   eval_every=20)

#### Plot the validation and training losses over (training) time

In [ ]:
### YOUR PLOTTING CODE HERE ### ==> DONE

plt.plot(range(len(val_l_g_sweep)), val_l_g_sweep)
plt.xticks(np.arange(len(val_l_g_sweep), step=2))
plt.xlabel('Training iteration')
plt.ylabel('Validation loss')
plt.title('Gradient descent validation loss, k = 30')
plt.show()

plt.plot(range(len(tr_l_g_sweep)), tr_l_g_sweep)
plt.xticks(np.arange(len(tr_l_g_sweep), step=2))
plt.xlabel('Training iteration')
plt.ylabel('Training loss')
plt.title('Gradient descent training loss, k = 30')
plt.show()

b) Learn the optimal $P$ and $Q$ using the original stochastic gradient descent (mini-batches of size 1). That is, during each iteration you sample a single random training example $r_{xi}$ and update only the respective affected parameters $\mathbf{p_x}$ and $\mathbf{q}_i$. Set the learning rate to 0.01 and keep the other parameters as in a).

In [ ]:
Q_g_st, P_g_st, val_l_g_st, tr_l_g_st, conv_g_st = latent_factor_gradient_descent(M_shifted, nonzero_indices, 
                                                                                   k=30, val_idx=val_idx,
                                                                                   val_values=val_values_shifted, 
                                                                                   reg_lambda=1e-1, learning_rate=1e-2,
                                                                                   init='svd', batch_size=1,
                                                                                   max_steps=20000, log_every=500, 
                                                                                   eval_every=50)

#### Plot the validation and training losses over (training) time

In [ ]:
### YOUR PLOTTING CODE HERE ### ==> DONE

plt.plot(range(len(val_l_g_st)), val_l_g_st)
plt.xticks(np.arange(len(val_l_g_st), step=2))
plt.xlabel('Training iteration')
plt.ylabel('Validation loss')
plt.title('Gradient descent validation loss, k = 30')
plt.show()

plt.plot(range(len(tr_l_g_st)), tr_l_g_st)
plt.xticks(np.arange(len(tr_l_g_st), step=2))
plt.xlabel('Training iteration')
plt.ylabel('Training loss')
plt.title('Gradient descent training loss, k = 30')
plt.show()

c) (**Optional**) Learn the optimal $P$ and $Q$ similarly to b) this time using larger mini-batches of size $S$, e.g. 32.

In [ ]:
Q_g_mb, P_g_mb, val_l_g_mb, tr_l_g_mb, conv_g_mb = latent_factor_gradient_descent(M_shifted, nonzero_indices, 
                                                                                   k=30, val_idx=val_idx,
                                                                                   val_values=val_values_shifted, 
                                                                                   reg_lambda=1, learning_rate=1e-1,
                                                                                   init='svd', batch_size=32,
                                                                                   max_steps=10000, log_every=100, 
                                                                                   eval_every=50)

#### Plot the validation and training losses over (training) time

In [ ]:
### YOUR PLOTTING CODE HERE ### ==> DONE

plt.plot(range(len(val_l_g_mb)), val_l_g_mb)
plt.xticks(np.arange(len(val_l_g_mb), step=2))
plt.xlabel('Training iteration')
plt.ylabel('Validation loss')
plt.title('Gradient descent validation loss, k = 30')
plt.show()

plt.plot(range(len(tr_l_g_mb)), tr_l_g_mb)
plt.xticks(np.arange(len(tr_l_g_mb), step=2))
plt.xlabel('Training iteration')
plt.ylabel('Training loss')
plt.title('Gradient descent training loss, k = 30')
plt.show()

### 4.5 Hyperparameter search

Machine learning models are often heavily dependent on the hyperparameter settings, e.g. the learning rate. Here, we will try a simple random search to find good values of the latent factor dimension $k$, the batch size, learning rate, and regularization.  

### Tasks:

Perform a hyperparameter search to find good values for the batch size, lambda, learning rate, and latent dimension. 

* For the batch size, evaluate all values in [1, 32, 512, -1] (-1 corresponds to full-sweep gradient descent).
* For $\lambda$, randomly sample three values in the interval [0, 1).
* For the learning rate, evaluate all values in [1, 0.1, 0.01].
* For the latent dimension, uniformly sample three values in the interval [5,30].

Perform an exhaustive search among all combinations of these values;

**Hint**: This may take a while to compute. **You don't have to wait for all the models to train** -- simply use "dummy" code instead of actual model training (or let it train, e.g., for only one iteration) if you don't want to wait. Note that the signature of this dummy code has to match the function 'latent_factor_gradient_descent' so that we could simply plug in the actual function.

### Students' comment:

**Note**: 
In order to get converging optimization the evaluated learning rates are adjusted to [1e-2, 1e-3, 1e-4]. <br> Furthermore, the range range of $\lambda$ is changed to [0, 0.1).

In [ ]:
def parameter_search(M_train, val_idx, val_values):
    """
    Hyperparameter search using random search.
    
    Parameters
    ----------
    
    M_train     : sp.spmatrix, shape [N, D]
                  Input sparse matrix where the user means have not
                  been subtracted yet. 
                  
    val_idx     : tuple, shape [2, n_validation]
                  The indices used for validation, where n_validation
                  is the size of the validation set.
                  
    val_values  : np.array, shape [n_validation, ]
                  Validation set values, where n_validation is the
                  size of the validation set.

    Returns
    -------
    best_conf   : tuple, (batch_size, lambda, learning_rate, latent_dimension)
                  The best-performing hyperparameters.
    
    best_Q      : np.array, shape [N, k]
                  Best value for Q (based on validation loss) observed during all configurations
                      
    best_P      : np.array, shape [k, D]
                  Best value for P (based on validation loss) observed during all configurations
    """
    
    ### YOUR CODE HERE ### ===> DONE
    
    # Store away the nonzero indices of M before substracting the row means.
    nonzero_indices =  M_train.nonzero()
    # Remove user means.
    M_shifted, _ = shift_user_mean(M_train)
    
    #Define the sets of hyperparameters
    batch_sizes = [1, 32, 512, -1]
    reg_lambdas = 0.1 * np.random.sample((3,))
    learning_rates = [1e-2, 1e-3, 1e-4]
    latent_dimensions = np.random.randint(5, 31, (3,))
    
    max_steps = 2
    
    #iterate over all combinations of hyperparameters
    for batch_size in batch_sizes:
        for reg_lambda in reg_lambdas:
            for learning_rate in learning_rates:
                for k in latent_dimensions:
                    
                    conf = (batch_size, reg_lambda, learning_rate, k)
                    print("Training with configuration {}".format(conf))
                    
                    #perform latent factor gradient descent for all combinations of hyperparameters
                    Q, P, val_l_g_opt, _, _ = latent_factor_gradient_descent(M_shifted, nonzero_indices, 
                                                                             k, val_idx=val_idx, 
                                                                             val_values=val_values_shifted, 
                                                                             reg_lambda = reg_lambda, learning_rate = learning_rate, 
                                                                             batch_size = batch_size, max_steps = max_steps, 
                                                                             init='svd', log_every=500, 
                                                                             eval_every=50)
                    
                    
                    #Get the best validation loss for the hyperparameters
                    min_val_l = min(val_l_g_opt)
                    print("Done. Best validation loss {}".format(min_val_l))
                    
                    
                    try:
                        #If the current hyperparameters lead to a better validation loss than all previous combinations
                        #before --> update the best configuration
                        if min_val_l < best_val_l:
                            best_val_l = min_val_l
                            best_conf = conf
                            best_Q = Q
                            best_P = P
                            print("New best configuration: {}".format(best_conf))
                            
                    #If first iteration: best_val_l is not defined yet and a NameError is raised
                    #Initialize best configuration as first configuration
                    except NameError:
                        best_val_l = min_val_l
                        best_conf = conf
                        best_Q = Q
                        best_P = P
                        print("New best configuration: {}".format(best_conf))
                        
                    print('\n')
  
    print("Best configuration is {}".format(best_conf))
    return best_conf, best_Q, best_P
    

In [ ]:
best_configuration, best_Q_g, best_P_g = parameter_search(M_train, val_idx, val_values)

#### Output the best hyperparameter optimization

In [ ]:
### YOUR CODE HERE ### ===> DONE

print("Best batch size: {}".format(best_configuration[0]))
print("Best lambda: {}".format(best_configuration[1]))
print("Best learning rate: {}".format(best_configuration[2]))
print("Best latent dimension: {}".format(best_configuration[3]))

## 5. Comparison of gradient descent and alternating optimization

After training the latent factor model with both alternating optimization and gradient descent, we now compare their results on the training, validation, and test set.

### Tasks

* Compare the root mean square errors (RMSE) for the training, validation, and test sets different settings of $k$ for both alternating optimization and gradient descent. What do you observe?
* Compare the test RMSE for the alternating optimization model and the gradient descent model. Which performs better?
* Plot the predicted ratings

**Hint**: The output values and plots below are the ones we got when testing this sheet. Yours may be different, but if your validation or test RMSE values are larger than 1.5 or 2, it is likely that you have a bug in your implementation.

In [ ]:
### YOUR CODE HERE ### ==> DONE

def get_shifted_predictions(indices, Q, P):
    shifted_values = np.sum(Q[indices[0]] * P.T[indices[1]], axis = 1)
    return shifted_values

def get_root_square_mean_error(true_values, prediction):
    return np.mean(np.square(true_values - prediction))

In [ ]:
### YOUR CODE HERE ### ==> DONE

train_values_a = M_shifted[nonzero_indices].A1
train_predicted_a = get_shifted_predictions(nonzero_indices, Q_a, P_a)
train_error_a = get_root_square_mean_error(train_values_a, train_predicted_a)
val_predicted_a = get_shifted_predictions(val_idx, Q_a, P_a)
val_error_a = get_root_square_mean_error(val_values_shifted, val_predicted_a)
test_predicted_a = get_shifted_predictions(test_idx, Q_a, P_a)
test_error_a = get_root_square_mean_error(test_values_shifted, test_predicted_a)

print("Training RMSE of alternating optimization model: {}".format(train_error_a))
print("Validation RMSE of alternating optimization model: {}".format(val_error_a))
print("Test RMSE of alternating optimization model: {}".format(test_error_a))

In [ ]:
### YOUR CODE HERE ### ==> DONE

train_values_g = M_shifted[nonzero_indices].A1
train_predicted_g = get_shifted_predictions(nonzero_indices, best_Q_g, best_P_g)
train_error_g = get_root_square_mean_error(train_values_g, train_predicted_g)
val_predicted_g = get_shifted_predictions(val_idx, best_Q_g, best_P_g)
val_error_g = get_root_square_mean_error(val_values_shifted, val_predicted_g)
test_predicted_g = get_shifted_predictions(test_idx, best_Q_g, best_P_g)
test_error_g = get_root_square_mean_error(test_values_shifted, test_predicted_g)

print("Training RMSE of best gradient descent model: {}".format(train_error_g))
print("Validation RMSE of best gradient descent model: {}".format(val_error_g))
print("Test RMSE of best gradient descent model: {}".format(test_error_g))

#### Plots: Prediction vs. ground truth ratings

In [ ]:
### YOUR PLOTTING CODE HERE ### ==> DONE

plt.scatter(train_values_a, train_predicted_a, 'MarkerFaceAlpha', 0.2)
plt.xlabel('Ground truth training rating')
plt.ylabel('Predicted rating')
plt.title('Training ratings vs. predictions by alternating optimization model')
plt.show()

In [ ]:
### YOUR PLOTTING CODE HERE ### ==> DONE
plt.scatter(train_values_g, train_predicted_g, 'MarkerFaceAlpha', 0.2)
plt.xlabel('Ground truth training rating')
plt.ylabel('Predicted rating')
plt.title('Training ratings vs. predictions by gradient descent model')
plt.show()

In [ ]:
### YOUR PLOTTING CODE HERE ### ==> DONE

plt.scatter(val_values_a, val_predicted_a, 'MarkerFaceAlpha', 0.2)
plt.xlabel('Ground truth training rating')
plt.ylabel('Predicted rating')
plt.title('Valdidation ratings vs. predictions by alternating optimization model')
plt.show()

In [ ]:
### YOUR PLOTTING CODE HERE ### ==> DONE

plt.scatter(train_values_g, train_predicted_g, 'MarkerFaceAlpha', 0.2)
plt.xlabel('Ground truth training rating')
plt.ylabel('Predicted rating')
plt.title('Validation ratings vs. predictions by gradient descent model')
plt.show()

In [ ]:
### YOUR PLOTTING CODE HERE ### ==> DONE

plt.scatter(test_values_a, test_predicted_a, 'MarkerFaceAlpha', 0.2)
plt.xlabel('Ground truth training rating')
plt.ylabel('Predicted rating')
plt.title('Test ratings vs. predictions by alternating optimization model')
plt.show()

In [ ]:
### YOUR PLOTTING CODE HERE ### ==> DONE

plt.scatter(test_values_g, test_predicted_g, 'MarkerFaceAlpha', 0.2)
plt.xlabel('Ground truth training rating')
plt.ylabel('Predicted rating')
plt.title('Test ratings vs. predictions by gradient descent model')
plt.show()